In [26]:
import codecs
import re
import warnings

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [27]:
stop_words = set(['de','la','que','el','en','y','a','los','del','se','las','por','un','para','con','no','una','su','al','lo','como','mas','pero','sus',
                     'le','ya','o','este','si','porque','esta','entre','cuando','muy','sin','sobre','tambien','me','hasta','hay','donde','quien','desde',
                     'todo','nos','durante','todos','uno','les','ni','contra','otros','ese','eso','ante','ellos','e','esto','mi','antes','algunos','que',
                     'unos','yo','otro','otras','otra','el','tanto','esa','estos','mucho','quienes','nada','muchos','cual','poco','ella','estar','estas',
                     'algunas','algo','nosotros','mi','mis','tu','te','ti','tu','tus','ellas','nosotras','vosotros','vosotras','os','mio','mia','mios',
                     'mias','tuyo','tuya','tuyos','tuyas','suyo','suya','suyos','suyas','nuestro','nuestra','nuestros','nuestras','vuestro','vuestra',
                     'vuestros','vuestras','esos','esas','estoy','estas','esta','estamos','estais','estan','este','estes','estemos','esteis','esten','estare',
                     'estaras','estara','estaremos','estareis','estaran','estaria','estarias','estariamos','estariais','estarian','estaba','estabas','estabamos',
                     'estabais','estaban','estuve','estuviste','estuvo','estuvimos','estuvisteis','estuvieron','estuviera','estuvieras','estuvieramos',
                     'estuvierais','estuvieran','estuviese','estuvieses','estuviesemos','estuvieseis','estuviesen','estando','estado','estada','estados','estadas',
                     'estad','he','has','ha','hemos','habeis','han','haya','hayas','hayamos','hayais','hayan','habre','habras','habra','habremos','habreis',
                     'habran','habria','habrias','habriamos','habriais','habrian','habia','habias','habiamos','habiais','habian','hube','hubiste','hubo','hubimos',
                     'hubisteis','hubieron','hubiera','hubieras','hubieramos','hubierais','hubieran','hubiese','hubieses','hubiesemos','hubieseis','hubiesen',
                     'habiendo','habido','habida','habidos','habidas','soy','eres','es','somos','sois','son','sea','seas','seamos','seais','sean','sere','seras',
                     'sera','seremos','sereis','seran','seria','serias','seriamos','seriais','serian','era','eras','eramos','erais','eran','fui','fuiste','fue',
                     'fuimos','fuisteis','fueron','fuera','fueras','fueramos','fuerais','fueran','fuese','fueses','fuesemos','fueseis','fuesen','sintiendo','sentido',
                     'sentida','sentidos','sentidas','siente','sentid','tengo','tienes','tiene','tenemos','teneis','tienen','tenga','tengas','tengamos','tengais',
                     'tengan','tendre','tendras','tendra','tendremos','tendreis','tendran','tendria','tendrias','tendriamos','tendriais','tendrian','tenia','tenias',
                     'teniamos','teniais','tenian','tuve','tuviste','tuvo','tuvimos','tuvisteis','tuvieron','tuviera','tuvieras','tuvieramos','tuvierais',
                     'tuvieran','tuviese','tuvieses','tuviesemos','tuvieseis','tuviesen','teniendo','tenido','tenida','tenidos','tenidas','tened', 'iii',
                     'iv', 'v', 'vi', 'vii', 'viii'])

# Obtiene el texto
def getText(file):
    text =[]
    with codecs.open('D:/Documentos/maestria_mcdi/3er semestre/NLP/docs/texto/' + file , 'r', encoding='utf8') as f:
        for line in f.readlines():
            clean_line = remove_esp_characters(line)
            if len(clean_line) > 0:
                text.append(clean_line)
    return text

#elimina caracteres especiales
def remove_esp_characters(txt):     
    text = re.sub(r'\W+',' ', txt)    
    text = re.sub(r'[ ]{2,}',' ', text)  
    return text.strip()

#clean texto
def clean(str):     
    txt = re.sub(r'[^a-zñ]', ' ', str)    
    txt = re.sub(r'[ ]{2,}',' ', txt)    
    text = re.sub(r'\n',' ', txt)       
    return text

# Remueve stop words
def remove_stop_words(str):      
    word = ""        
    text = str.split()        
    for _text in text:        
        if _text not in stop_words:
            #Agrega al diccionario palabras que tenga 2 caracteres en adelante
            if(len(clean(_text)) > 2):                
                word += _text  + " " 
            
    return word 

In [28]:
"""
Archivos creados en el proceso de creación de words embeddings se eliminó puntuación y acentos 

text_vf sin pre-procesamiento
text_lemma_vf  Se aplico proceso de lematización
text_stemming_vf Se aplico proceso de stemming

"""
files = ["text_vf.txt" , "text_lemma_vf.txt", "text_stemming_vf.txt"]

list_text = []
for f in files:
    list_text.append(" ".join(getText(f)))


# LDA con proceso de lematización

In [29]:
def get_vectorizer(_text):
    # Vectorizar TF
    vectorizer = CountVectorizer(stop_words=list(stop_words))
    X_vec_TF = vectorizer.fit_transform(_text)

    # Vectorizar TF-IDF
    tfidfvec = TfidfVectorizer(stop_words=list(stop_words))
    X_vec_TFIDF = tfidfvec.fit_transform(_text)
    
    X_vec_dense_TF = X_vec_TF.todense()
    X_vec_dense_TFIDF = X_vec_TFIDF.todense()
    
    print("Sparsicity TF: ", ((X_vec_dense_TF > 0).sum()/X_vec_dense_TF.size)*100, "%")
    print("Sparsicity TFIDF: ", ((X_vec_dense_TFIDF > 0).sum()/X_vec_dense_TFIDF.size)*100, "%")
    
    return X_vec_TF, vectorizer, X_vec_TFIDF, tfidfvec

def get_LDA(X_vec_TF, X_vec_TFIDF, _components):        
    lda_model = LatentDirichletAllocation(n_components=_components,
                                      max_iter=10,
                                      random_state=100,
                                      batch_size=128,
                                      evaluate_every = -1,
                                      n_jobs = -1
                                     )
    lda_tf = lda_model.fit_transform(X_vec_TF)
    lda_tfidf = lda_model.fit_transform(X_vec_TFIDF)
    
    print("Log Likelihood TF: ", lda_model.score(X_vec_TF))
    print("Log Likelihood TFIDF: ", lda_model.score(X_vec_TFIDF))

    print("Perplexity TF: ", lda_model.perplexity(X_vec_TF))
    print("Perplexity TFIDF: ", lda_model.perplexity(X_vec_TF))
    
    return lda_model
        
    

## Tópicos con counter Vectorizer

In [30]:
text_ssp_lemma = remove_stop_words(list_text[1])

In [31]:
X_vec_TF, vectorizer, X_vec_TFIDF, tfidfvec = get_vectorizer([text_ssp_lemma])
lda_model = get_LDA(X_vec_TF, X_vec_TFIDF, 30)

Sparsicity TF:  100.0 %
Sparsicity TFIDF:  100.0 %
Log Likelihood TF:  -68626668.98206554
Log Likelihood TFIDF:  -608.9552899085048
Perplexity TF:  85233107075.57246
Perplexity TFIDF:  85233107075.57246


In [33]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TF, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  99.999960
29      54.382130  109.799164       2        1   0.000001
28    -161.423965 -103.744957       3        1   0.000001
1     -135.370758   36.673065       4        1   0.000001
3      177.317215  -66.872452       5        1   0.000001
4      -16.436356  142.453796       6        1   0.000001
5       31.052383  204.777573       7        1   0.000001
6      -25.024384 -186.083466       8        1   0.000001
7      -68.157486   20.934038       9        1   0.000001
8       -3.751695    4.961509      10        1   0.000001
9     -114.470009 -179.605469      11        1   0.000001
10    -110.729126  -40.476776      12        1   0.000001
11      42.552418  -42.789406      13        1   0.000001
12     -10.661536   71.922211      14        1   0.000001
13     112.652969  -16.279951      15        1   0.000001
14    -195.499146  -25.853945      16        1   0.000001
15    -207.165924   66.070419      17        1   0.000001
16      10.217649 -110.752060      18        1   0.000001
17     -81.255081   96.477776      19        1   0.000001
18     -77.253441  192.755219      20        1   0.000001
19     182.353027  110.412880      21        1   0.000001
20      89.766510  -93.997673      22        1   0.000001
21     -36.483528  -52.542774      23        1   0.000001
22     -72.052170 -112.144821      24        1   0.000001
23     144.822769 -150.811508      25        1   0.000001
24    -148.475357  135.678162      26        1   0.000001
25     189.811615   17.382956      27        1   0.000001
26      56.555355   33.755768      28        1   0.000001
27      58.988537 -178.376541      29        1   0.000001
0      110.556717  164.236694      30        1   0.000001, topic_info=           Term         Freq        Total Category  logprob  loglift
13508     parir  1451.000000  1451.000000  Default  30.0000  30.0000
1571   articulo  1416.000000  1416.000000  Default  29.0000  29.0000
16954       ser  1122.000000  1122.000000  Default  28.0000  28.0000
5232      deber   978.000000   978.000000  Default  27.0000  27.0000
11111       ley   949.000000   949.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1        100mgs     0.000002   131.737649  Topic30  -9.9112   0.0266
5        115bis     0.000002   131.737649  Topic30  -9.9112   0.0266
16          1er     0.000002   131.760881  Topic30  -9.9112   0.0264
18          1gr     0.000002   131.737649  Topic30  -9.9112   0.0266
19          1ro     0.000002   131.745373  Topic30  -9.9112   0.0266

[1075 rows x 6 columns], token_table=       Topic      Freq                          Term
term                                                
0          1  1.002050  083ac178388e5d9b38128352d42a
1          1  1.001991                        100mgs
2          1  1.002050                       100xqi2
3          1  1.002050                          10el
4          1  1.002050                       10estos
...      ...       ...                           ...
15312      1  0.999228                      realizar
15615      1  1.000427                       referir
16759      1  0.999865                    secretaria
16954      1  1.000004                           ser
17292      1  1.000155                      sociedad

[65 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])

## Tópicos con counter TFIDF Vectorizer

In [35]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TFIDF, tfidfvec, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  94.966519
29      54.382130  109.799164       2        1   0.173568
28    -161.423965 -103.744957       3        1   0.173568
1     -135.370758   36.673065       4        1   0.173568
3      177.317215  -66.872452       5        1   0.173568
4      -16.436356  142.453796       6        1   0.173568
5       31.052383  204.777573       7        1   0.173568
6      -25.024384 -186.083466       8        1   0.173568
7      -68.157486   20.934038       9        1   0.173568
8       -3.751695    4.961509      10        1   0.173568
9     -114.470009 -179.605469      11        1   0.173568
10    -110.729126  -40.476776      12        1   0.173568
11      42.552418  -42.789406      13        1   0.173568
12     -10.661536   71.922211      14        1   0.173568
13     112.652969  -16.279951      15        1   0.173568
14    -195.499146  -25.853945      16        1   0.173568
15    -207.165924   66.070419      17        1   0.173568
16      10.217649 -110.752060      18        1   0.173568
17     -81.255081   96.477776      19        1   0.173568
18     -77.253441  192.755219      20        1   0.173568
19     182.353027  110.412880      21        1   0.173568
20      89.766510  -93.997673      22        1   0.173568
21     -36.483528  -52.542774      23        1   0.173568
22     -72.052170 -112.144821      24        1   0.173568
23     144.822769 -150.811508      25        1   0.173568
24    -148.475357  135.678162      26        1   0.173568
25     189.811615   17.382956      27        1   0.173568
26      56.555355   33.755768      28        1   0.173568
27      58.988537 -178.376541      29        1   0.173568
0      110.556717  164.236694      30        1   0.173568, topic_info=           Term      Freq     Total Category  logprob  loglift
13508     parir  0.000000  0.000000  Default  30.0000  30.0000
1571   articulo  0.000000  0.000000  Default  29.0000  29.0000
16954       ser  0.000000  0.000000  Default  28.0000  28.0000
5232      deber  0.000000  0.000000  Default  27.0000  27.0000
11111       ley  0.000000  0.000000  Default  26.0000  26.0000
...         ...       ...       ...      ...      ...      ...
1        100mgs  0.000002  0.001002  Topic30  -9.9112   0.0253
5        115bis  0.000002  0.001002  Topic30  -9.9112   0.0253
16          1er  0.000002  0.001002  Topic30  -9.9112   0.0251
18          1gr  0.000002  0.001002  Topic30  -9.9112   0.0253
19          1ro  0.000002  0.001002  Topic30  -9.9112   0.0252

[1075 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])

# LDA con proceso de Stemming

## Tópicos con counter Vectorizer

In [36]:
text_ssp_stemm = remove_stop_words(list_text[2])

In [16]:
X_vec_TF, vectorizer, X_vec_TFIDF, tfidfvec = get_vectorizer([text_ssp_stemm])
lda_model = get_LDA(X_vec_TF, X_vec_TFIDF, 30)

Sparsicity TF:  100.0 %
Sparsicity TFIDF:  100.0 %
Log Likelihood TF:  -65720194.867064625
Log Likelihood TFIDF:  -573.4606574998336
Perplexity TF:  24493193796.518078
Perplexity TFIDF:  24493193796.518078


In [37]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TF, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  99.999960
29      54.382130  109.799164       2        1   0.000001
28    -161.423965 -103.744957       3        1   0.000001
1     -135.370758   36.673065       4        1   0.000001
3      177.317215  -66.872452       5        1   0.000001
4      -16.436356  142.453796       6        1   0.000001
5       31.052383  204.777573       7        1   0.000001
6      -25.024384 -186.083466       8        1   0.000001
7      -68.157486   20.934038       9        1   0.000001
8       -3.751695    4.961509      10        1   0.000001
9     -114.470009 -179.605469      11        1   0.000001
10    -110.729126  -40.476776      12        1   0.000001
11      42.552418  -42.789406      13        1   0.000001
12     -10.661536   71.922211      14        1   0.000001
13     112.652969  -16.279951      15        1   0.000001
14    -195.499146  -25.853945      16        1   0.000001
15    -207.165924   66.070419      17        1   0.000001
16      10.217649 -110.752060      18        1   0.000001
17     -81.255081   96.477776      19        1   0.000001
18     -77.253441  192.755219      20        1   0.000001
19     182.353027  110.412880      21        1   0.000001
20      89.766510  -93.997673      22        1   0.000001
21     -36.483528  -52.542774      23        1   0.000001
22     -72.052170 -112.144821      24        1   0.000001
23     144.822769 -150.811508      25        1   0.000001
24    -148.475357  135.678162      26        1   0.000001
25     189.811615   17.382956      27        1   0.000001
26      56.555355   33.755768      28        1   0.000001
27      58.988537 -178.376541      29        1   0.000001
0      110.556717  164.236694      30        1   0.000001, topic_info=           Term         Freq        Total Category  logprob  loglift
13508     parir  1451.000000  1451.000000  Default  30.0000  30.0000
1571   articulo  1416.000000  1416.000000  Default  29.0000  29.0000
16954       ser  1122.000000  1122.000000  Default  28.0000  28.0000
5232      deber   978.000000   978.000000  Default  27.0000  27.0000
11111       ley   949.000000   949.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1        100mgs     0.000002   131.737649  Topic30  -9.9112   0.0266
5        115bis     0.000002   131.737649  Topic30  -9.9112   0.0266
16          1er     0.000002   131.760881  Topic30  -9.9112   0.0264
18          1gr     0.000002   131.737649  Topic30  -9.9112   0.0266
19          1ro     0.000002   131.745373  Topic30  -9.9112   0.0266

[1075 rows x 6 columns], token_table=       Topic      Freq                          Term
term                                                
0          1  1.002050  083ac178388e5d9b38128352d42a
1          1  1.001991                        100mgs
2          1  1.002050                       100xqi2
3          1  1.002050                          10el
4          1  1.002050                       10estos
...      ...       ...                           ...
15312      1  0.999228                      realizar
15615      1  1.000427                       referir
16759      1  0.999865                    secretaria
16954      1  1.000004                           ser
17292      1  1.000155                      sociedad

[65 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])

## Tópicos con counter TFIDF Vectorizer

In [38]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TFIDF, tfidfvec, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  94.966519
29      54.382130  109.799164       2        1   0.173568
28    -161.423965 -103.744957       3        1   0.173568
1     -135.370758   36.673065       4        1   0.173568
3      177.317215  -66.872452       5        1   0.173568
4      -16.436356  142.453796       6        1   0.173568
5       31.052383  204.777573       7        1   0.173568
6      -25.024384 -186.083466       8        1   0.173568
7      -68.157486   20.934038       9        1   0.173568
8       -3.751695    4.961509      10        1   0.173568
9     -114.470009 -179.605469      11        1   0.173568
10    -110.729126  -40.476776      12        1   0.173568
11      42.552418  -42.789406      13        1   0.173568
12     -10.661536   71.922211      14        1   0.173568
13     112.652969  -16.279951      15        1   0.173568
14    -195.499146  -25.853945      16        1   0.173568
15    -207.165924   66.070419      17        1   0.173568
16      10.217649 -110.752060      18        1   0.173568
17     -81.255081   96.477776      19        1   0.173568
18     -77.253441  192.755219      20        1   0.173568
19     182.353027  110.412880      21        1   0.173568
20      89.766510  -93.997673      22        1   0.173568
21     -36.483528  -52.542774      23        1   0.173568
22     -72.052170 -112.144821      24        1   0.173568
23     144.822769 -150.811508      25        1   0.173568
24    -148.475357  135.678162      26        1   0.173568
25     189.811615   17.382956      27        1   0.173568
26      56.555355   33.755768      28        1   0.173568
27      58.988537 -178.376541      29        1   0.173568
0      110.556717  164.236694      30        1   0.173568, topic_info=           Term      Freq     Total Category  logprob  loglift
13508     parir  0.000000  0.000000  Default  30.0000  30.0000
1571   articulo  0.000000  0.000000  Default  29.0000  29.0000
16954       ser  0.000000  0.000000  Default  28.0000  28.0000
5232      deber  0.000000  0.000000  Default  27.0000  27.0000
11111       ley  0.000000  0.000000  Default  26.0000  26.0000
...         ...       ...       ...      ...      ...      ...
1        100mgs  0.000002  0.001002  Topic30  -9.9112   0.0253
5        115bis  0.000002  0.001002  Topic30  -9.9112   0.0253
16          1er  0.000002  0.001002  Topic30  -9.9112   0.0251
18          1gr  0.000002  0.001002  Topic30  -9.9112   0.0253
19          1ro  0.000002  0.001002  Topic30  -9.9112   0.0252

[1075 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])

# LDA sin pre-procesamientos

## Tópicos con counter Vectorizer

In [19]:
X_vec_TF, vectorizer, X_vec_TFIDF, tfidfvec = get_vectorizer([list_text[0]])
lda_model = get_LDA(X_vec_TF, X_vec_TFIDF, 30)

Sparsicity TF:  100.0 %
Sparsicity TFIDF:  100.0 %
Log Likelihood TF:  -77280918.21004155
Log Likelihood TFIDF:  -856.7168282222799
Perplexity TF:  818936935440.143
Perplexity TFIDF:  818936935440.143


In [39]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TF, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  99.999960
29      54.382130  109.799164       2        1   0.000001
28    -161.423965 -103.744957       3        1   0.000001
1     -135.370758   36.673065       4        1   0.000001
3      177.317215  -66.872452       5        1   0.000001
4      -16.436356  142.453796       6        1   0.000001
5       31.052383  204.777573       7        1   0.000001
6      -25.024384 -186.083466       8        1   0.000001
7      -68.157486   20.934038       9        1   0.000001
8       -3.751695    4.961509      10        1   0.000001
9     -114.470009 -179.605469      11        1   0.000001
10    -110.729126  -40.476776      12        1   0.000001
11      42.552418  -42.789406      13        1   0.000001
12     -10.661536   71.922211      14        1   0.000001
13     112.652969  -16.279951      15        1   0.000001
14    -195.499146  -25.853945      16        1   0.000001
15    -207.165924   66.070419      17        1   0.000001
16      10.217649 -110.752060      18        1   0.000001
17     -81.255081   96.477776      19        1   0.000001
18     -77.253441  192.755219      20        1   0.000001
19     182.353027  110.412880      21        1   0.000001
20      89.766510  -93.997673      22        1   0.000001
21     -36.483528  -52.542774      23        1   0.000001
22     -72.052170 -112.144821      24        1   0.000001
23     144.822769 -150.811508      25        1   0.000001
24    -148.475357  135.678162      26        1   0.000001
25     189.811615   17.382956      27        1   0.000001
26      56.555355   33.755768      28        1   0.000001
27      58.988537 -178.376541      29        1   0.000001
0      110.556717  164.236694      30        1   0.000001, topic_info=           Term         Freq        Total Category  logprob  loglift
13508     parir  1451.000000  1451.000000  Default  30.0000  30.0000
1571   articulo  1416.000000  1416.000000  Default  29.0000  29.0000
16954       ser  1122.000000  1122.000000  Default  28.0000  28.0000
5232      deber   978.000000   978.000000  Default  27.0000  27.0000
11111       ley   949.000000   949.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1        100mgs     0.000002   131.737649  Topic30  -9.9112   0.0266
5        115bis     0.000002   131.737649  Topic30  -9.9112   0.0266
16          1er     0.000002   131.760881  Topic30  -9.9112   0.0264
18          1gr     0.000002   131.737649  Topic30  -9.9112   0.0266
19          1ro     0.000002   131.745373  Topic30  -9.9112   0.0266

[1075 rows x 6 columns], token_table=       Topic      Freq                          Term
term                                                
0          1  1.002050  083ac178388e5d9b38128352d42a
1          1  1.001991                        100mgs
2          1  1.002050                       100xqi2
3          1  1.002050                          10el
4          1  1.002050                       10estos
...      ...       ...                           ...
15312      1  0.999228                      realizar
15615      1  1.000427                       referir
16759      1  0.999865                    secretaria
16954      1  1.000004                           ser
17292      1  1.000155                      sociedad

[65 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])

## Tópicos con counter TFIDF Vectorizer

In [40]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_vec_TFIDF, tfidfvec, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      119.070190   64.655502       1        1  94.966519
29      54.382130  109.799164       2        1   0.173568
28    -161.423965 -103.744957       3        1   0.173568
1     -135.370758   36.673065       4        1   0.173568
3      177.317215  -66.872452       5        1   0.173568
4      -16.436356  142.453796       6        1   0.173568
5       31.052383  204.777573       7        1   0.173568
6      -25.024384 -186.083466       8        1   0.173568
7      -68.157486   20.934038       9        1   0.173568
8       -3.751695    4.961509      10        1   0.173568
9     -114.470009 -179.605469      11        1   0.173568
10    -110.729126  -40.476776      12        1   0.173568
11      42.552418  -42.789406      13        1   0.173568
12     -10.661536   71.922211      14        1   0.173568
13     112.652969  -16.279951      15        1   0.173568
14    -195.499146  -25.853945      16        1   0.173568
15    -207.165924   66.070419      17        1   0.173568
16      10.217649 -110.752060      18        1   0.173568
17     -81.255081   96.477776      19        1   0.173568
18     -77.253441  192.755219      20        1   0.173568
19     182.353027  110.412880      21        1   0.173568
20      89.766510  -93.997673      22        1   0.173568
21     -36.483528  -52.542774      23        1   0.173568
22     -72.052170 -112.144821      24        1   0.173568
23     144.822769 -150.811508      25        1   0.173568
24    -148.475357  135.678162      26        1   0.173568
25     189.811615   17.382956      27        1   0.173568
26      56.555355   33.755768      28        1   0.173568
27      58.988537 -178.376541      29        1   0.173568
0      110.556717  164.236694      30        1   0.173568, topic_info=           Term      Freq     Total Category  logprob  loglift
13508     parir  0.000000  0.000000  Default  30.0000  30.0000
1571   articulo  0.000000  0.000000  Default  29.0000  29.0000
16954       ser  0.000000  0.000000  Default  28.0000  28.0000
5232      deber  0.000000  0.000000  Default  27.0000  27.0000
11111       ley  0.000000  0.000000  Default  26.0000  26.0000
...         ...       ...       ...      ...      ...      ...
1        100mgs  0.000002  0.001002  Topic30  -9.9112   0.0253
5        115bis  0.000002  0.001002  Topic30  -9.9112   0.0253
16          1er  0.000002  0.001002  Topic30  -9.9112   0.0251
18          1gr  0.000002  0.001002  Topic30  -9.9112   0.0253
19          1ro  0.000002  0.001002  Topic30  -9.9112   0.0252

[1075 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 30, 29, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1])